In [16]:
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.signal as sg
import sounddevice as sd
import wavio

In [17]:
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sg


In [18]:
def mult_by_exp(samples, Fc, Fs):
    return samples*np.exp(-1j * (np.arange(0, len(samples)) * 2 * np.pi * Fc / Fs))

In [19]:
def fix_diff(sample):
    res = np.zeros(len(sample))
    for i in range(len(sample)):
        if sample[i] > np.pi:
            res[i] = sample[i] - 2 * np.pi
        elif sample[i] < -np.pi:
            res[i] = sample[i] + 2 * np.pi
        else:
            res[i] = sample[i]
    return res

In [20]:
def amplify(sample, div, Fs):
    return sample * Fs/div

In [21]:
def receiver(samples, div, Fs, Fc):
    s1 = mult_by_exp(samples, Fc, Fs)
    b,a = sg.iirdesign(4000, min(2*Fc, Fs-2*Fc)-4000, 1, 100, fs=Fs)
    s2 = sg.lfilter(b, a, s1)
    s3 = np.angle(s2)
    s4 = np.diff(s3)
    s5 = fix_diff(s4.copy())
    s6 = amplify(s5, div, Fs)/2/np.pi
    return s6

In [22]:
def transmitter(sample, div, Fs, Fc):
    s = np.zeros(len(sample))
    time = np.arange(0, len(sample))
    integral = 0
    A0 = 1
    for t in time:
        s[t] = A0 * np.real(np.exp(1j * (2 * np.pi * t * Fc / Fs + div * integral)))
        integral += sample[t] / Fs
    return s

In [23]:
# samples, Fs = sf.read('123.ogg')
# Fc = 7000
# div = 1e3
# s = transmitter(samples, div, Fs, Fc)
# out = receiver(s, div, Fs, Fc)
# wavio.write("sig3.mp3", out, Fs, sampwidth=2)

In [24]:
import time, math, sys
import scipy.signal as sig
import scipy.fftpack as fft
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import sounddevice as sd

In [25]:
class Filter(object):
    def __init__(self, b, a = [1], init=[]):
        self.b = b
        self.a = a
        self.clear(init)
        
    def __call__(self, x):
        y, self.state = sig.lfilter(self.b, self.a, x, zi=self.state)
        return y
    
    def clear(self, init=[]):
        self.state = sig.lfiltic(self.b, self.a, [], init)

In [26]:
diff = Filter([1, -1])
def fix_diff(x):
    x[x > math.pi] -= 2 * math.pi
    x[x < -math.pi] += 2 * math.pi
    return x

In [27]:
def block_reciever(samples, div, Fs, Fc, last_ph):
    s1 = mult_by_exp(samples, Fc, Fs, last_ph)
    b,a = sg.iirdesign(4000, min(2*Fc, Fs-2*Fc)-4000, 1, 100, fs=Fs)
    s2 = sg.lfilter(b, a, s1)
    s3 = np.angle(s2)
    s4 = diff(s3)
    s5 = fix_diff(s4.copy())
    s6 = amplify(s5, div, Fs)/2/np.pi
    return s6

In [28]:
samples, Fs = sf.read('encoded.flac')
N = int(len(samples) / 1000)
#s = block_receiver(samples[:1000], 1e3, Fs, 7000)
last_ph = 0
Fc = 7000
div = 1e3

lpf = Filter(*sg.iirdesign(4000, min(2*Fc, Fs-2*Fc)-4000, 1, 100, fs=Fs))
s = np.array([])
for i in range(0,N-1):
    samp = samples[i*1000:(i+1)*1000]
    s1 = mult_by_exp(samp, Fc, Fs)*np.exp(-1j * last_ph)
    s2 = lpf(s1)
    s3 = np.angle(s2)
    s4 = diff(s3)
    s5 = fix_diff(s4.copy())
    s6 = amplify(s5, div, Fs)/2/np.pi
    s = np.concatenate([s, s6])
    last_ph = last_ph + len(samp) * 2 * np.pi * Fc / Fs
print(len(s))


3423000


In [29]:
wavio.write("sig4.mp3", s, Fs, sampwidth=2)